****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [1]:
import pandas as pd
import numpy as np
from xml.etree import ElementTree as ET

In [2]:
document = ET.parse( './data/mondial_database.xml' )

In [3]:
counties=[]
mortRates=[]
for element in document.iterfind('country'):
    counties.append(element.find('name').text)
    capitals_string = ''
    for subelement in element.getiterator('country'):
        try: capitals_string += subelement.find('infant_mortality').text + ', '
        except:pass
    mortRates.append(capitals_string[:-2])
df=pd.DataFrame()
df['contries']=counties
df['mortRates']=mortRates
df.loc[df.mortRates.str.len() == 0,'mortRates'] = np.nan
df['mortRates']=df['mortRates'].astype(float)
df.sort('mortRates')[:10]

,contries,mortRates
38,Monaco,1.81
98,Japan,2.13
117,Bermuda,2.48
36,Norway,2.48
106,Singapore,2.53
37,Sweden,2.60
10,Czech Republic,2.63
78,Hong Kong,2.73
79,Macao,3.13
44,Iceland,3.15


##10 cities with the largest population

In [4]:
cities=[]
population=[]
df=pd.DataFrame()
for city in document.findall('country/city'):
    #name = ''
    #population = 0
    for node in list(city):
        if node.tag == 'name':
            name = node.text
        elif node.tag == 'population':
            # note: if multiple population tags, the last is used
            # population tags sorted by year, so last is most recent
            pop = int(node.text)
    # only add cities with known populations
    if population != 0:
        cities.append(name)
        population.append(pop)

df['cities']=cities
df['population']=population
df.sort('population', ascending=False)[:10]

,cities,population
176,Seoul,9708483
164,Cairo,8471859
80,Bangkok,7506700
129,Macao,7055071
128,Hong Kong,7055071
92,Ho Chi Minh City,5968384
212,Singapore,5076700
163,Alexandria,4123869
216,New Taipei,3939305
177,Pusan,3403135


##10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)

In [5]:
ethnGroup=[]
population=[]
df=pd.DataFrame()
# extract country, ethnic group, and population from xml
for country in document.findall('country'):
    #name = ''
    #group = ''
    #population = 0
    for node in list(country):
        if node.tag == 'name':
            name = node.text
        elif node.tag == 'population':
            # note: if multiple population tags, the last is used
            # population tags sorted by year, so last is most recent
            pop = int(node.text)
            year = int(node.attrib['year'])
        elif node.tag == 'ethnicgroup':
            percentage = float(node.attrib['percentage'])
            group = node.text
            
            if group and percentage and pop:
            # only add groups with known populations
                #country.append(name)
                ethnGroup.append(group)
                # calculate ethnic group population from percentage of overall population
                population.append(int(pop * percentage / 100))
df['ethnGroup']=ethnGroup
df['population']=population
df.sort('population', ascending=False)[:10]

,ethnGroup,population
176,Han Chinese,1245058800
221,Indo-Aryan,871815583
220,Dravidian,302713744
345,European,254958101
520,African,162651570
212,Bengali,146776916
299,Japanese,126534212
93,Russian,114646210
278,Javanese,113456006
461,European,108886717


##name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [6]:
# create dictionary for country codes
countries=[]
codes=[]
countriesDf = pd.DataFrame()
for country in document.findall('country'):
    for node in list(country):
        if node.tag == 'name':           
            countries.append(node.text)
            codes.append(country.attrib['car_code'])
countriesDf['countries'] = countries
countriesDf['codes'] = codes
countriesDf.set_index('codes',inplace=True)
dictionary=countriesDf.to_dict('dict').get('countries')

##Longest River

In [7]:
rows = []
for river in document.iterfind('river'):
    try:
        rows.append({'name':river.find("name").text,'length':int(river.find('length').text),'country':river.find('located').attrib['country']})
        df = pd.DataFrame(rows) 
    except:
        pass
df.replace( {"country": dictionary}, inplace=True)
df.sort('length', ascending=False)[:1]

,country,length,name
161,Colombia,6448,Amazonas


##Largest Lake

In [8]:
rows = []
for lake in document.iterfind('lake'):
    try:
        rows.append({'name':lake.find("name").text,'area':int(lake.find('area').text),'country':lake.find('located').attrib['country']})
        df = pd.DataFrame(rows) 
    except:
        pass
df.replace( {"country": dictionary}, inplace=True)
df.sort('area', ascending=False)[:1]

,area,country,name
42,386400,Russia,Caspian Sea


##Highest Airport

In [9]:
rows = []
for airport in document.iterfind('airport'):
    try:
        rows.append({'name':airport.find("name").text,'elevation':int(airport.find('elevation').text),'country':airport.attrib['country']})
        df = pd.DataFrame(rows) 
    except:
        pass
df.replace( {"country": dictionary}, inplace=True)
df.sort('elevation', ascending=False)[:1]

,country,elevation,name
80,Bolivia,4063,El Alto Intl
